In [1]:
import pickle
import os

from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargo Dataset principal

In [2]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [ ]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'BOS_2') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100),
        lags=14
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)
    
    # Predecir
    predicciones = forecaster.predict(steps=30, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    print(f"Predicciones generadas para el producto: {producto}")


In [4]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,27,18.0,0
1,HOME_&_GARDEN_1_027,43,51.0,0
2,HOME_&_GARDEN_1_053,115,107.0,0
3,HOME_&_GARDEN_1_140,64,62.0,0
4,HOME_&_GARDEN_1_177,34,64.0,0
...,...,...,...,...
95,SUPERMARKET_3_499,388,394.0,0
96,ACCESORIES_1_158,229,258.0,0
97,SUPERMARKET_3_282,758,488.0,0
98,ACCESORIES_1_354,298,347.0,0


In [5]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

27640
29315.0
-1675.0


## Predicción

In [3]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'BOS_2') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100),
        lags=14
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    print(f"Predicciones generadas para el producto: {producto}")

Predicciones generadas para el producto: ACCESORIES_1_108
Predicciones generadas para el producto: HOME_&_GARDEN_1_027
Predicciones generadas para el producto: HOME_&_GARDEN_1_053
Predicciones generadas para el producto: HOME_&_GARDEN_1_140


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: HOME_&_GARDEN_1_177
Predicciones generadas para el producto: HOME_&_GARDEN_1_272
Predicciones generadas para el producto: HOME_&_GARDEN_1_344
Predicciones generadas para el producto: HOME_&_GARDEN_1_373
Predicciones generadas para el producto: HOME_&_GARDEN_1_440
Predicciones generadas para el producto: HOME_&_GARDEN_1_537
Predicciones generadas para el producto: SUPERMARKET_1_096
Predicciones generadas para el producto: SUPERMARKET_2_019


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: SUPERMARKET_2_029
Predicciones generadas para el producto: SUPERMARKET_2_052


...

Predicciones generadas para el producto: SUPERMARKET_3_362
Predicciones generadas para el producto: SUPERMARKET_3_499


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: ACCESORIES_1_158
Predicciones generadas para el producto: SUPERMARKET_3_282


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: ACCESORIES_1_354
Predicciones generadas para el producto: HOME_&_GARDEN_1_106


In [4]:
tienda =  pd.DataFrame(resultados)

In [5]:
file_name = "BOS_2_unidades.xlsx"
tienda.to_excel(file_name)